In [2]:
import pandas as pd 
import yfinance as yf
import requests
from io import StringIO
import re

In [3]:
def load_nasdaq_tickers()-> pd.DataFrame:
    url = "https://www.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt"
    response = requests.get(url, timeout=30)
    lines = response.text.strip().split("\n") 
    lines = lines[:-1]  # Remove the last line which is a footer
    df = pd.read_csv(StringIO("\n".join(lines)),sep="|")
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    print(len(df), "before filtering")
    df = df[df["test_issue"] == "N"]
    df = df[df["etf"] == "N"]
    print(len(df), "after etf/test filtering")
    exclude_pattern = re.compile(r"unit|right|warrant|trust|fund|etf|etn|depositary", re.IGNORECASE)
    df = df[~df["security_name"].str.contains(exclude_pattern, na=False)]
    print(len(df), "after name filtering")
    df = df[df["security_name"].str.contains("common stock", case=False, na=False)]
    print(len(df), "after common stock filtering")
    print(len(df[df["market_category"] == "Q"]), "after market category filtering")
    return df

load_nasdaq_tickers()

5282 before filtering
4154 after etf/test filtering
3228 after name filtering
2332 after common stock filtering
1069 after market category filtering


,symbol,security_name,market_category,test_issue,financial_status,round_lot_size,etf,nextshares
6,AAL,"American Airlines Group, Inc. - Common Stock",Q,N,N,100,N,N
8,AAME,Atlantic American Corporation - Common Stock,G,N,N,100,N,N
9,AAOI,"Applied Optoelectronics, Inc. - Common Stock",G,N,N,100,N,N
10,AAON,"AAON, Inc. - Common Stock",Q,N,N,100,N,N
14,AAPL,Apple Inc. - Common Stock,Q,N,N,100,N,N
...,...,...,...,...,...,...,...,...
5267,ZSPC,"zSpace, Inc. - Common stock",S,N,D,100,N,N
5268,ZTEK,Zentek Ltd. - common stock,S,N,D,100,N,N
5273,ZUMZ,Zumiez Inc. - Common Stock,Q,N,N,100,N,N
5275,ZVRA,"Zevra Therapeutics, Inc. - Common Stock",Q,N,N,100,N,N


In [4]:
def load_nyse_tickers() -> pd.DataFrame:
    url = "https://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt"
    response = requests.get(url, timeout=30)
    response.raise_for_status()
    lines = response.text.strip().split("\n") 
    lines = lines[:-1]  # Remove the last line which is a footer
    df = pd.read_csv(StringIO("\n".join(lines)),sep="|")
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    
    print(len(df), "before filtering")
    df = df[df["exchange"].isin(["N", "A", "P"])]
    print(len(df), "after filtering for exchange")
    df = df[df["etf"] == "N"]
    print(len(df), "after etf filtering")
    df = df[df["test_issue"] == "N"]
    print(len(df), "after test issue filtering")
    df = df[df["security_name"].str.contains("common stock", case=False, na=False)]
    print(len(df), "after common stock filtering")
    return df
load_nyse_tickers()

6963 before filtering
5759 after filtering for exchange
3156 after etf filtering
3137 after test issue filtering
1808 after common stock filtering


,act_symbol,security_name,exchange,cqs_symbol,etf,round_lot_size,test_issue,nasdaq_symbol
0,A,"Agilent Technologies, Inc. Common Stock",N,A,N,100,N,A
1,AA,Alcoa Corporation Common Stock,N,AA,N,100,N,AA
9,AAMI,Acadian Asset Management Inc. Common Stock,N,AAMI,N,100,N,AAMI
15,AAT,"American Assets Trust, Inc. Common Stock",N,AAT,N,100,N,AAT
19,ABBV,AbbVie Inc. Common Stock,N,ABBV,N,100,N,ABBV
...,...,...,...,...,...,...,...,...
6938,ZIP,"ZipRecruiter, Inc. Class A Common Stock",N,ZIP,N,100,N,ZIP
6947,ZONE,CleanCore Solutions Inc. Class B Common Stock,A,ZONE,N,100,N,ZONE
6957,ZTS,Zoetis Inc. Class A Common Stock,N,ZTS,N,100,N,ZTS
6958,ZVIA,Zevia PBC Class A Common Stock,N,ZVIA,N,100,N,ZVIA


In [5]:
def load_euronext_tickers() -> pd.DataFrame:
    url = "https://live.euronext.com/pd_es/data/stocks/download"
    params = {
        "mics": "dm_all_stock",
        "issueType": "101", # Common Stocks
        "initialLetter": "",
        "display_datapoints": (
            "logo,name,isin,symbol,market,"
            "lastPrice,precentDayChange,lastTradeTime"
        ),
        "fe_type": "csv",
        "fe_decimal_separator": ".",
        "fe_date_format": "d/m/Y",
    }
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "text/csv",
    }

    # --- Download ---
    response = requests.get(
        url,
        params=params,
        headers=headers
    )
    response.raise_for_status()

    csv_text = response.text

    # --- CSV korrekt einlesen ---
    df = pd.read_csv(
        StringIO(csv_text),
        sep=";",              # Euronext nutzt Semikolon
        encoding="utf-8-sig", # robust gegen BOM
        low_memory=False,
    )
    df =df[3:] # Erste 3 Zeilen sind Metadaten
    MARKET_TO_YAHOO_SUFFIX = {
    # --- Paris ---
    "Euronext Paris": ".PA",
    "Euronext Growth Paris": ".PA",
    "Euronext Access Paris": ".PA",

    # --- Amsterdam ---
    "Euronext Amsterdam": ".AS",

    # --- Brussels ---
    "Euronext Brussels": ".BR",
    "Euronext Growth Brussels": ".BR",
    "Euronext Access Brussels": ".BR",

    # --- Milan ---
    "Euronext Milan": ".MI",
    "Euronext Growth Milan": ".MI",

    # --- Lisbon ---
    "Euronext Lisbon": ".LS",
    "Euronext Growth Lisbon": ".LS",
    "Euronext Access Lisbon": ".LS",

    # --- Dublin ---
    "Euronext Dublin": ".IR",
    "Euronext Growth Dublin": ".IR",
    "Euronext Access Dublin": ".IR",

    # --- Oslo ---
    "Oslo Børs": ".OL",
    "Euronext Growth Oslo": ".OL",
    "Euronext Expand Oslo": ".OL",

    # --- Multilistings (mappe auf Primary Listing) ---
    "Euronext Brussels, Paris": ".BR",
    "Euronext Paris, Brussels": ".PA",
    "Euronext Brussels, Amsterdam": ".BR",
    "Euronext Amsterdam, Brussels": ".AS",
    "Euronext Amsterdam, Paris": ".AS",
    "Euronext Paris, Amsterdam": ".PA",
    "Euronext Paris, Amsterdam, Brussels": ".PA",
    "Euronext Amsterdam, Brussels, Paris": ".AS",
    "Euronext Brussels, Amsterdam, Paris": ".BR",
    "Euronext Growth Brussels, Paris": ".BR",
    "Euronext Growth Paris, Brussels": ".PA",
    }
    df = df[df['Market'].isin(MARKET_TO_YAHOO_SUFFIX.keys())] # Entferne Euronext Global Equity Market, Trading After Hours, EuroTLX da kein Primärlisting
    def build_yahoo_ticker(symbol: str, market: str) -> str:
        suffix = MARKET_TO_YAHOO_SUFFIX.get(market, "")
        return f"{symbol}{suffix}"
    df["YahooTicker"] = df.apply(lambda row: build_yahoo_ticker(row["Symbol"], row["Market"]), axis=1)

    return df
data = load_euronext_tickers()
data

,Name,ISIN,Symbol,Market,Currency,Open Price,High Price,low Price,last Price,last Trade MIC Time,Time Zone,Volume,Turnover,Closing Price,Closing Price DateTime,YahooTicker
3,2020 BULKERS,BMG9156K1018,2020,Oslo Børs,NOK,144.20,144.20,140.70,141.70,28/01/2026 16:26,CET,148628,21060984.00,141.70,28/01/2026,2020.OL
4,2CRSI,FR0013341781,AL2SI,Euronext Growth Paris,EUR,13.62,13.94,12.60,12.78,28/01/2026 17:35,CET,395015,5235618.18,12.78,28/01/2026,AL2SI.PA
11,4AIM SICAF,IT0005204729,AIM,Euronext Growth Milan,EUR,69.80,72.00,69.80,72.00,22/01/2026 15:34,CET,36,2522.00,72.00,28/01/2026,AIM.MI
12,4AIM SICAF COMP 2,IT0005440323,AIM2,Euronext Growth Milan,EUR,131.50,131.50,131.50,131.50,14/01/2026 17:11,CET,2,263.00,131.50,28/01/2026,AIM2.MI
13,5TH PLANET GAMES,DK0060945467,5PG,Euronext Expand Oslo,NOK,0.88,0.88,0.84,0.858,28/01/2026 16:06,CET,42291,35942.242,0.858,28/01/2026,5PG.OL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,ZELLUNA,NO0013524942,ZLNA,Oslo Børs,NOK,14.70,15.00,14.50,14.602,28/01/2026 16:15,CET,8337,121934.068,14.602,28/01/2026,ZLNA.OL
3812,ZENITH ENERGY,CA98936C8584,ZENA,Euronext Growth Oslo,NOK,0.47,0.484,0.455,0.467,28/01/2026 16:25,CET,7198879,3362254.887,0.467,28/01/2026,ZENA.OL
3813,ZEST,IT0005013013,ZEST,Euronext Milan,EUR,0.1475,0.1475,0.1425,0.1455,28/01/2026 17:06,CET,124700,17931.8275,0.1455,28/01/2026,ZEST.MI
3814,ZIGNAGO VETRO,IT0004171440,ZV,Euronext Milan,EUR,7.62,7.68,7.52,7.65,28/01/2026 17:35,CET,83749,639729.17,7.65,28/01/2026,ZV.MI


In [ ]:
nasdaq_t = load_nasdaq_tickers()["symbol"]
nyse_t = load_nyse_tickers()["cqs_symbol"]
euronext_t = load_euronext_tickers()["YahooTicker"]
tickers =pd.concat([nasdaq_t, nyse_t, euronext_t]).to

5282 before filtering
4154 after etf/test filtering
3228 after name filtering
2332 after common stock filtering
1069 after market category filtering
6963 before filtering
5759 after filtering for exchange
3156 after etf filtering
3137 after test issue filtering
1808 after common stock filtering


In [9]:
tickers.to_csv("tickers.csv", index=False)